In [4]:
#finetune bert model for multilaabel classification

# import libraries
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
#from transformers import BertTokenizer, BertModel, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix


# load data
combined_df = pd.read_csv('combined_df.csv')
combined_df.head()

,Index,BookTitle,Author,Summary,Genres,clean_summary
0,1,Animal Farm,George Orwell,"Old Major, the old boar on the Manor Farm, ca...","['Roman à clef', 'Satire', ""Children's literat...",old major the old boar on the manor farm calls...
1,2,A Clockwork Orange,Anthony Burgess,"Alex, a teenager living in near-future Englan...","['Science Fiction', 'Novella', 'Speculative fi...",alex a teenager living in near future england ...
2,3,The Plague,Albert Camus,The text of The Plague is divided into five p...,"['Existentialism', 'Fiction', 'Absurdist ficti...",the text of the plague is divided into five pa...
3,5,A Fire Upon the Deep,Vernor Vinge,The novel posits that space around the Milky ...,"['Hard science fiction', 'Science Fiction', 'S...",the novel posits that space around the milky w...
4,6,All Quiet on the Western Front,Erich Maria Remarque,"The book tells the story of Paul Bäumer, a Ge...","['War novel', 'Roman à clef']",the book tells the story of paul b umer a germ...


In [22]:
genre_df = pd.read_csv('genre_df.csv')
genre_df.head()

,11th_century,17th_century,19th_century,20th_century,21st_century,40k,_school_stories,absurdist_fiction,abuse,academia,...,x_men,yeti,young_adult,young_adult_contemporary,young_adult_fantasy,young_adult_literature,youth,zombie,zombies,zombies_in_popular_culture
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
#convert each row to a list 
genre_lists = []
for i in range(len(genre_df)):
    #put row into a list
    genre_list = genre_df.iloc[i].tolist()
    genre_lists.append(genre_list)


#create mapping from genres in genre_df columns to index
genre_to_index = {}
index_to_genre = {}
for i in range(len(genre_df.columns)):
    genre_to_index[genre_df.columns[i]] = i
    index_to_genre[i] = genre_df.columns[i]







In [29]:
combined_df['genre_list'] = genre_lists
combined_df.head()

,Index,BookTitle,Author,Summary,Genres,clean_summary,genre_list
0,1,Animal Farm,George Orwell,"Old Major, the old boar on the Manor Farm, ca...","['Roman à clef', 'Satire', ""Children's literat...",old major the old boar on the manor farm calls...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,2,A Clockwork Orange,Anthony Burgess,"Alex, a teenager living in near-future Englan...","['Science Fiction', 'Novella', 'Speculative fi...",alex a teenager living in near future england ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,3,The Plague,Albert Camus,The text of The Plague is divided into five p...,"['Existentialism', 'Fiction', 'Absurdist ficti...",the text of the plague is divided into five pa...,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
3,5,A Fire Upon the Deep,Vernor Vinge,The novel posits that space around the Milky ...,"['Hard science fiction', 'Science Fiction', 'S...",the novel posits that space around the milky w...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,6,All Quiet on the Western Front,Erich Maria Remarque,"The book tells the story of Paul Bäumer, a Ge...","['War novel', 'Roman à clef']",the book tells the story of paul b umer a germ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
#Custom Dataset class
#accepts tokenizer, dataframe, and max length
#BERT tokenizer is used to tokenize the text

max_len = 512
train_batch_size = 16
valid_batch_size = 16
epochs = 10
learning_rate = 1e-05
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


In [33]:
class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.summary = dataframe.clean_summary
        self.targets = self.data.genre_list
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs['token_type_ids']

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [ ]:
#create dataset and data loader
train_size = 0.8
train_dataset=combined_df.sample(frac=train_size,random_state=200)
test_dataset=combined_df.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

training_set = CustomDataset(train_dataset, tokenizer, max_len)
testing_set = CustomDataset(test_dataset, tokenizer, max_len)

In [ ]:
training_params = {
    'batch_size': train_batch_size,
    'shuffle': True,
    'num_workers': 0
}

testing_params = {
    'batch_size': valid_batch_size,
    'shuffle': True,
    'num_workers': 0
}

training_loader = DataLoader(training_set, **training_params)
testing_loader = DataLoader(testing_set, **testing_params)

In [ ]:
#BERT model
num_classes = len(genre_df.columns)

class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l1 = BertModel.from_pretrained('bert-base-uncased')
        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Linear(768, num_classes)

    def forward(self, ids, mask, token_type_ids):
        _, output_1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids)
        output_2 = self.l2(output_1)
        output = self.l3(output_2)
        return output
    
model = BERTClass()
model.to(device)


In [ ]:
#import metrics
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
#unfreeze last 4 layers
for param in model.l1.base_model.encoder.layer[-4:].parameters():
    param.requires_grad = True #unfreeze last 4 layers so they can be updated during training
 
#loss function and optimizer
loss_function = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=learning_rate)

#train model
for epoch in range(epochs):
    model.train()
    train_loss = 0
    for _,data in enumerate(training_loader, 0):
        #put data on gpu
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)


        outputs = model(ids, mask, token_type_ids)
        loss = loss_function(outputs, targets)
        train_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    #validation
    model.eval()
    valid_loss = 0
    valid_preds = []
    valid_labels = []
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            #put data on gpu
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)

            outputs = model(ids, mask, token_type_ids)
            loss = loss_function(outputs, targets)
            valid_loss += loss.item()

            valid_preds.append(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
            valid_labels.append(targets.cpu().detach().numpy().tolist())

    print(f'Epoch: {epoch}, Loss:  {train_loss/len(training_loader)}')
    print(f'Epoch: {epoch}, Valid Loss:  {valid_loss/len(testing_loader)}')
    #print accuracy
    
    
    #calculate accuracy
    valid_preds = np.array(valid_preds) >= 0.5
    accuracy = accuracy_score(valid_labels, valid_preds)
    print(f'Epoch: {epoch}, Valid Accuracy:  {accuracy}')

    #print f1 score
    f1 = f1_score(valid_labels, valid_preds, average='micro')
    print(f'Epoch: {epoch}, Valid F1 Score:  {f1}')




In [ ]:
#get cls embedding from bert
#intialize empty tensor to store cls embeddings


#will this actually work??????

full_data = CustomDataset(combined_df, tokenizer, max_len)
full_loader = DataLoader(full_data, **testing_params)

all_cls_embeddings = torch.empty(len(full_data), 768).to(device)

with torch.no_grad():
    for i, data in enumerate(full_loader, 0):
        #put data on gpu
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model.l1(ids, attention_mask = mask, token_type_ids = token_type_ids)
        cls_embeddings = outputs[0][:, 0, :]
        all_cls_embeddings[i*train_batch_size:(i+1)*train_batch_size, :] = cls_embeddings



